In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext fortranmagic

/home/simone/anaconda3/envs/trisonica-anemometer-comparison/lib/python3.7/site-packages/fortranmagic.py:147: UserWarning: get_ipython_cache_dir has moved to the IPython.paths module since IPython 4.0.
  self._lib_dir = os.path.join(get_ipython_cache_dir(), 'fortran')


In [ ]:
import numpy as np

In [ ]:
%%fortran

subroutine f1(x, y, z)
    real, intent(in) :: x,y
    real, intent(out) :: z

    z = sin(x+y)

end subroutine f1

In [ ]:
f1(2,3)

-0.9589242935180664

In [ ]:
from wind_tools import *

fpath = Path("test_data/20180809-2030_WM_174605_com1.raw")

In [ ]:
n_samples = 30 / 5

df = pd.read_csv(fpath, usecols=(0,2,3,4,6), names=['tstmp','u', 'v', 'w', 't'], header=None)

In [ ]:
df

,tstmp,u,v,w,t
0,203000,0.13,0.20,0.20,28.33
1,203000,0.06,0.18,0.16,28.35
2,203000,0.05,0.19,0.15,28.38
3,203000,0.10,0.18,0.20,28.43
4,203000,0.23,0.13,0.22,28.47
...,...,...,...,...,...
18001,205959,0.43,0.55,0.18,27.52
18002,205959,0.32,0.69,-0.04,27.50
18003,205959,0.36,0.54,-0.08,27.52
18004,205959,0.53,0.48,-0.28,27.50


In [ ]:
proc_dir = Path("/run/media/simone/Simone DATI/TRISONICA_DATA/Processed/")
wm1_path = proc_dir / "WM1_ago_sept" / "eddypro_WM1_full_output_2019-12-20T105458_exp.csv"

df.groupby('tstmp').count().query('u == 9').count()

df.groupby(pd.cut(df.index, n_samples)).mean()

wm1 = load_ep_cache(wm1_path).rename(columns=wind_comp_rename)

rot = rotate_u_v(df, -310)

rotg = rot.groupby(pd.cut(df.index, n_samples)).mean()

In [ ]:
m = df.mean()

In [ ]:
np.mean(df)

tstmp    204479.857770
u             0.247699
v             0.310869
w            -0.016067
t            27.946013
dtype: float64

In [ ]:
df.mean().T

tstmp    204479.857770
u             0.247699
v             0.310869
w            -0.016067
t            27.946013
dtype: float64

In [ ]:
df = add_wind_speed(df)

In [ ]:
df.max()

tstmp         210000.000000
u                  1.960000
v                  1.590000
w                  1.020000
t                 28.470000
wind_speed         2.595804
dtype: float64

In [ ]:
df.mean()

tstmp         204479.857770
u                  0.247699
v                  0.310869
w                 -0.016067
t                 27.946013
wind_speed         0.757498
dtype: float64

In [ ]:
df.mean().index

Index(['tstmp', 'u', 'v', 'w', 't', 'wind_speed'], dtype='object')

In [ ]:
df.mean().to_numpy()

array([ 2.04479858e+05,  2.47699100e-01,  3.10868599e-01, -1.60668666e-02,
        2.79460130e+01,  7.57497780e-01])

In [ ]:
m = pd.DataFrame(df.mean().to_numpy()[None], columns=df.mean().index)

In [ ]:
add_wind_dir(m)

warning add_wind_dir is not the same of EddyPro


,tstmp,u,v,w,t,wind_speed,wind_dir
0,204479.85777,0.247699,0.310869,-0.016067,27.946013,0.757498,198.357909


In [ ]:
add_wind_speed(m)

,tstmp,u,v,w,t,wind_speed
0,204479.85777,0.247699,0.310869,-0.016067,27.946013,0.397809


In [ ]:
%%fortran --extra "-I/home/simone/Downloads/applications/eddypro-engine/src/src_common" --extra "-I/home/simone/Downloads/applications/eddypro-engine/obj/linux"



!***************************************************************************
! wind_direction.f90
! ------------------
! Copyright (C) 2007-2011, Eco2s team, Gerardo Fratini
! Copyright (C) 2011-2019, LI-COR Biosciences, Inc.  All Rights Reserved.
! Author: Gerardo Fratini
!
! This file is part of EddyPro®.
!
! NON-COMMERCIAL RESEARCH PURPOSES ONLY - EDDYPRO® is licensed for 
! non-commercial academic and government research purposes only, 
! as provided in the EDDYPRO® End User License Agreement. 
! EDDYPRO® may only be used as provided in the End User License Agreement
! and may not be used or accessed for any commercial purposes.
! You may view a copy of the End User License Agreement in the file
! EULA_NON_COMMERCIAL.rtf.
!
! Commercial companies that are LI-COR flux system customers 
! are encouraged to contact LI-COR directly for our commercial 
! EDDYPRO® End User License Agreement.
!
! EDDYPRO® contains Open Source Components (as defined in the 
! End User License Agreement). The licenses and/or notices for the 
! Open Source Components can be found in the file LIBRARIES-ENGINE.txt.
!
! EddyPro® is distributed in the hope that it will be useful,
! but WITHOUT ANY WARRANTY; without even the implied warranty of
! MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
!
!***************************************************************************
!
! \brief       Calculates wind direction from single wind components pair and offset
! \brief       It's essentially a vector direction + offset
! \author      Gerardo Fratini
! \note
! \sa
! \bug
! \deprecated
! \test
! \todo
!***************************************************************************
subroutine SingleWindDirection(Wind, offset, WindDir)
    use m_common_global_var
    implicit none
    !> in/out variables
    real(kind = dbl), intent(in) :: Wind(3)
    real(kind = dbl), intent(in) :: offset
    real(kind = dbl), intent(out) :: WindDir


    if (Wind(U) /= error .and. Wind(V) /= error)  then 
        !> Calculate raw wind direction from wind vector
        WindDir = 180 - (datan2(Wind(V), Wind(U)) * 180d0 / p)

        !> accounts for user-supplied anemometer mis-alignment
        WindDir = WindDir + offset

        !> wrap within 0 - 360
        if (WindDir >= 360d0) WindDir = WindDir - 360d0
        if (WindDir < 0d0)   WindDir = 360d0 + WindDir
    else
        WindDir = error
    end if
end subroutine SingleWindDirection

!***************************************************************************
!
! \brief       Calculates mean wind direction and compensates offset
! \author      Gerardo Fratini
! \note        
! \sa
! \bug
! \deprecated
! \test
! \todo
!***************************************************************************
subroutine AverageWindDirection(Set, nrow, ncol, offset, WindDir, err_float)
    use m_common_global_var
    implicit none
    !> in/out variables
    integer, intent(in) :: nrow, ncol
    real(kind = dbl), intent(in) :: err_float
    real(kind = dbl), intent(in) :: Set(nrow, ncol)
    real(kind = dbl), intent(in) :: offset
    real(kind = dbl), intent(out) :: WindDir
    !> Local variables
    real(kind = dbl):: wd(nrow)
    integer :: i


    !> Compute raw-level wind-direction
    do i = 1, nrow
        call SingleWindDirection(Set(i, u:w), offset, wd(i))
    end do
    
    !> Compute mean wind direction
    call AngularAverageNoError(wd, nrow, 1, WindDir, err_float)

end subroutine AverageWindDirection


!***************************************************************************
!
! \brief       Calculates standard deviation of wind direction
! \author      Gerardo Fratini
! \note        
! \sa
! \bug
! \deprecated
! \test
! \todo
!***************************************************************************
subroutine WindDirectionStDev(Set, nrow, ncol, WindDirStDev, err_float)
    use m_common_global_var
    implicit none
    !> in/out variables
    integer, intent(in) :: nrow, ncol
    real(kind = dbl), intent(in) :: err_float
    real(kind = dbl), intent(in) :: Set(nrow, ncol)
    real(kind = dbl), intent(out) :: WindDirStDev
    !> Local variables
    real(kind = dbl):: wd(nrow)
    integer :: i


    !> Compute raw-level wind-direction
    do i = 1, nrow
        call SingleWindDirection(Set(i, u:w), 0d0, wd(i))
    end do
    
    !> Compute mean wind direction
    call AngularStDevApproxNoError(wd, nrow, 1, WindDirStDev, err_float)

end subroutine WindDirectionStDev


!***************************************************************************
!
! \brief       Calculate column-wise averages on a 2d array \n
!              ignoring specified error values \n
! \author      Gerardo Fratini
! \note
! \sa
! \bug
! \deprecated
! \test
! \todo
!***************************************************************************
subroutine AverageNoError(Set, nrow, ncol, Mean, err_float)
    use m_common_global_var
    implicit none
    !> in/out variables
    integer, intent(in) :: nrow, ncol
    real(kind = dbl), intent(in) :: err_float
    real(kind = dbl), intent(in) :: Set(nrow, ncol)
    real(kind = dbl), intent(out) :: Mean(ncol)
    !> local variables
    integer :: i = 0
    integer :: j = 0
    integer :: Nact = 0
    real(kind = dbl) :: RawMean(ncol)


    RawMean = 0d0
    do j = 1, ncol
        Nact = 0
        do i = 1, nrow
            if (Set(i, j) /= err_float) then
                Nact = Nact + 1
                RawMean(j) = RawMean(j) + Set(i, j)
            end if
        end do
        if (Nact /= 0) then
            RawMean(j) = RawMean(j) / dble(Nact)
        else
            RawMean(j) = err_float
        end if
    end do
    Mean = 0.d0
    do j = 1, ncol
        if (RawMean(j) /= err_float) then
            Nact = 0
            do i = 1, nrow
                if (Set(i, j) /= err_float) then
                    Nact = Nact + 1
                    Mean(j) = Mean(j) + Set(i, j) - RawMean(j)
                end if
            end do
            if (Nact /= 0) then
                Mean(j) = Mean(j) / dble(Nact)
            else
                Mean(j) = err_float
            end if
        else
            Mean(j) = err_float
        end if
    end do

    where (Mean(:) /= err_float)
        Mean(:) = Mean(:) + RawMean(:)
    elsewhere
        Mean(:) = err_float
    end where
end subroutine AverageNoError

!***************************************************************************
!
! \brief       Calculate column-wise angular averages on a 2d array \n
!              ignoring specified error values. In EddyPro, mainly meant for \n
!              calculation of mean wind direction given a set of wind direction \n
!              measurements.
!
!              Implementation reference:
!              "Circular Statistics in R"
!              by A. Pewsey, M. Neuhaeuser and G. D. Ruxton.
!              Yamartino, 1984:
!              https://journals.ametsoc.org/doi/pdf/10.1175/1520-0450%281984%29023%3C1362%3AACOSPE%3E2.0.CO%3B2
! \author      Gerardo Fratini
! \note
! \sa
! \bug
! \deprecated
! \test
! \todo
!***************************************************************************
subroutine AngularAverageNoError(Set, nrow, ncol, Mean, err_float)
    use m_common_global_var
    implicit none
    !> in/out variables
    integer, intent(in) :: nrow, ncol
    real(kind = dbl), intent(in) :: err_float
    real(kind = dbl), intent(in) :: Set(nrow, ncol)
    real(kind = dbl), intent(out) :: Mean(ncol)
    !> local variables
    integer :: i = 0
    integer :: j = 0
    integer :: Nact = 0
    real(kind = dbl) :: CosSum
    real(kind = dbl) :: SinSum


    do j = 1, ncol

        !> Calculate a (CosSum) and b (SinSum)
        CosSum = 0d0
        SinSum = 0d0
        Nact = 0
        do i = 1, nrow
            if (Set(i, j) /= err_float) then
                Nact = Nact + 1
                CosSum = CosSum - dcos(Set(i, j) / 180d0 * p)
                SinSum = SinSum - dsin(Set(i, j) / 180d0 * p)
            end if
        end do
        if (Nact /= 0) then
            CosSum = CosSum / dble(Nact)
            SinSum = SinSum / dble(Nact)
        else
            Mean(j) = err_float
            cycle
        end if

        !> Angular average is atan2 of b and a
        !> "+p" adjust quadrant, then express in degrees
        Mean(j) = (datan2(SinSum, CosSum) + p) * 180d0 / p
    end do
end subroutine AngularAverageNoError

!***************************************************************************
!
! \brief       Calculate column-wise angular stdev on a 2d array \n
!              ignoring specified error values. In EddyPro, mainly meant for \n
!              calculation of wind direction standard deviation given a set of wind direction. \n
!
!              Implementation reference:
!              Yamartino, 1984:
!              https://journals.ametsoc.org/doi/pdf/10.1175/1520-0450%281984%29023%3C1362%3AACOSPE%3E2.0.CO%3B2
! \author      Gerardo Fratini
! \note
! \sa
! \bug
! \deprecated
! \test
! \todo
!***************************************************************************
subroutine AngularStDevApproxNoError(Set, nrow, ncol, AngStDev, err_float)
    use m_common_global_var
    implicit none
    !> in/out variables
    integer, intent(in) :: nrow, ncol
    real(kind = dbl), intent(in) :: err_float
    real(kind = dbl), intent(in) :: Set(nrow, ncol)
    real(kind = dbl), intent(out) :: AngStDev(ncol)
    !> local variables
    integer :: i = 0
    integer :: j = 0
    integer :: Nact = 0
    real(kind = dbl) :: CosSum
    real(kind = dbl) :: SinSum
    real(kind = dbl) :: eps


    do j = 1, ncol

        !> Calculate a (CosSum) and b (SinSum)
        CosSum = 0d0
        SinSum = 0d0
        Nact = 0
        do i = 1, nrow
            if (Set(i, j) /= err_float) then
                Nact = Nact + 1
                CosSum = CosSum - dcos(Set(i, j) / 180d0 * p)
                SinSum = SinSum - dsin(Set(i, j) / 180d0 * p)
            end if
        end do
        if (Nact /= 0) then
            CosSum = CosSum / dble(Nact)
            SinSum = SinSum / dble(Nact)
        else
            AngStDev(j) = err_float
            cycle
        end if

        !> Approximate standard deviation of wind direction (see Yamartino, 1984:
        !> https://journals.ametsoc.org/doi/pdf/10.1175/1520-0450%281984%29023%3C1362%3AACOSPE%3E2.0.CO%3B2)
        eps = dsqrt(1. - (CosSum**2 + SinSum**2))
        AngStDev = (dasin(eps) * (1. + (2 / sqrt(3.) - 1) * eps**3)) * 180d0 / p
    end do
end subroutine AngularStDevApproxNoError

   

In [ ]:
singlewinddirection(np.array([0,0,0]),50)

0.0

In [ ]:
def ang_mean(ar):
    x = np.cos(np.deg2rad(ar))
    y = np.cos(np.deg2rad(ar))
    return np.arctan2(y.sum(), x.sum())
    

In [ ]:
ta = np.array([30,0,10])
angularaveragenoerror(np.array(ta), 0.001)

array([13.29504673])

In [ ]:
ang_mean(ta)

0.7853981633974483